In [1]:
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

2023-05-24 00:18:46.468198: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 00:18:46.637701: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 00:18:46.638436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 00:18:47.762650: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!ls

In [ ]:
!python model_main_tf2.py --model_dir=output/ --pipeline_config_path=ssd_mobilenet_v2_ft/pipeline.config

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir src/output/ --output_directory src/ssd_mobilenet_v2_ft2

In [18]:
PATH_TO_MODEL_DIR = 'ssd_mobilenet_v2_ft/'
PATH_TO_LABELS = '/home/irizqy/ml_ws/bangkit-ws/data/label_map.pbtxt'

In [19]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 7.963697195053101 seconds


In [16]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [25]:
import numpy as np
import os

rand_im = np.random.choice(os.listdir('/home/irizqy/ml_ws/bangkit-ws/data/temp'), 5)
rand_im


array(['10_20230507_095726.jpg', '17_20230507_092258.jpg',
       "Carrabba's Italian Grill-44.jpg", '24_20230510_161358.jpg',
       "Carrabba's Italian Grill-44.jpg"], dtype='<U31')

In [26]:
BASED_PATH = '/home/irizqy/ml_ws/bangkit-ws/data/temp'
IMAGE_PATHS = [os.path.join(BASED_PATH, im) for im in rand_im]
IMAGE_PATHS

['/home/irizqy/ml_ws/bangkit-ws/data/temp/10_20230507_095726.jpg',
 '/home/irizqy/ml_ws/bangkit-ws/data/temp/17_20230507_092258.jpg',
 "/home/irizqy/ml_ws/bangkit-ws/data/temp/Carrabba's Italian Grill-44.jpg",
 '/home/irizqy/ml_ws/bangkit-ws/data/temp/24_20230510_161358.jpg',
 "/home/irizqy/ml_ws/bangkit-ws/data/temp/Carrabba's Italian Grill-44.jpg"]

In [27]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
count = 0

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)
    
    if image_np.shape[2] == 4:
        image_np = image_np[:, :, :3]

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    # print(detections)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    im = Image.fromarray(image_np_with_detections)
    im.save(f"detected-logo2{count}.jpeg")
    count+=1
    print('Done')
    plt.show()

# sphinx_gallery_thumbnail_number = 2

Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/10_20230507_095726.jpg... defaultdict(<class 'str'>, {})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/17_20230507_092258.jpg... defaultdict(<class 'str'>, {(0.3763657212257385, 0.3562638759613037, 0.5385758280754089, 0.8214502334594727): 'Aqua', (0.36441770195961, 0.13157489895820618, 0.513726532459259, 0.33116307854652405): 'Chartreuse'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/Carrabba's Italian Grill-44.jpg... defaultdict(<class 'str'>, {(0.32878565788269043, 0.5741132497787476, 0.41798990964889526, 0.6940445899963379): 'Aqua'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/24_20230510_161358.jpg... defaultdict(<class 'str'>, {(0.4506187438964844, 0.05816996097564697, 0.6414238214492798, 0.896354615688324): 'Aqua'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/temp/Carrabba's Italian Grill-44.jpg... defaultdict(<class 'str'>, {(0.328785

In [ ]:
import cv2 as cv

In [ ]:
# print(detections['detection_scores'])
# print(detections['detection_boxes'])
# print(detections['detection_classes'])
detections.items()

In [ ]:
width, height, _ = image_np.shape

raw_detection_boxes <br>
detection_scores<br>
detection_boxes<br>
detection_anchor_indices<br>
raw_detection_scores<br>
detection_classes<br>
detection_multiclass_scores<br>
num_detections

In [ ]:
ymin, xmin, ymax, xmax = detections['detection_boxes'][0]
(left, right, top, bottom) = (xmin*width, xmax*width, ymin*height, ymax*height)
print(int(left), int(right), int(top), int(bottom))

In [ ]:
im_top = cv.circle(image_np, (int(top), int(top+right)), 2, (0, 266, 0), 2)
# im_bot = 
# im_right = 
# im_lrft = 

Image.fromarray(im_top).save('top.jpg')

In [ ]:
im = cv.rectangle(image_np, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 3)
Image.fromarray(im).save('test.jpg')

In [ ]:
cropped_im = image_np[int(top):int(bottom), int(left):int(right)]
Image.fromarray(cropped_im).save('cropped-logo.jpg')
